# COVID-19 CASES COMPARISON: LOGARITHMIC VS ACTUAL SCALE
This Jupyter Notebook visualizes the evolution of COVID-19 cases across different countries from 2020 to 2023. 
We use both logarithmic and actual scales to compare case growth.

In [ ]:
!pip install pandas 
!pip install plotly 
!pip install numpy

In [ ]:
import pandas as pd
import json
import plotly.express as px
import numpy as np
import plotly.io as pio
from plotly.subplots import make_subplots

In [ ]:
geojson_path = 'https://raw.githubusercontent.com/N1kN4me/covid19-interactive/dcee83fa6e74c64f694f9acfef49f858ca566ece/countries.geojson'
csv_path = 'https://raw.githubusercontent.com/N1kN4me/covid19-interactive/dcee83fa6e74c64f694f9acfef49f858ca566ece/covid19_dataset_merged.csv'

# Load GeoJSON and CSV Data
geojson_data = pd.read_json(geojson_path)
covid_data = pd.read_csv(csv_path)

# Group by country and sum across dates
covid_grouped = covid_data.groupby('Country/Region').sum(numeric_only=True)
date_columns = [col for col in covid_grouped.columns if '/' in col or '-' in col]
covid_grouped = covid_grouped[date_columns].transpose()
covid_grouped.index = pd.to_datetime(covid_grouped.index, errors='coerce').dropna().strftime('%Y-%m-%d')
covid_grouped.reset_index(inplace=True)

# Transform to long format
covid_transposed = covid_grouped.melt(id_vars=['index'], var_name='Country', value_name='Cases')
covid_transposed.rename(columns={'index': 'Date'}, inplace=True)

# Apply logarithmic transformation to Cases
covid_transposed['Log_Cases'] = np.log1p(covid_transposed['Cases'])  # log(Cases + 1) to avoid log(0)

max_cases = covid_transposed['Cases'].max()
log_max = covid_transposed['Log_Cases'].max()
log_colors = [
    "#ffffcc", "#fff8bb", "#fff0a9", "#fee997", "#fee186", "#fed875", "#feca65", "#feba54",
    "#fdaa48", "#fd9b42", "#fc8c3b", "#fc7434", "#fc5a2d", "#f64327", "#ec2d21", "#e2191c",
    "#d30f20", "#c40424", "#af0026", "#960026", "#800026"
]

#Log Scale Map
fig1 = px.choropleth(
    covid_transposed,
    geojson=geojson_data,
    locations='Country',
    featureidkey='properties.name',
    color='Log_Cases',
    animation_frame='Date',
    color_continuous_scale=log_colors,
    range_color=(0, log_max),
    title='COVID-19 Cases Evolution (Logarithmic Scale) 2020 - 2023',
    labels={'Log_Cases': 'Logarithmic Cases (log1p)'}
)
fig1.update_traces(marker_line_width=1, marker_line_color='black')
fig1.update_geos(fitbounds="locations", visible=True, showcountries=True)

# Show Logarithmic Scale Map
fig1.show()

#Actual Scale Map
fig2 = px.choropleth(
    covid_transposed,
    geojson=geojson_data,
    locations='Country',
    featureidkey='properties.name',
    color='Cases',
    animation_frame='Date',
    color_continuous_scale=log_colors,
    range_color=(0, max_cases),
    title='COVID-19 Cases Evolution (Actual Scale) 2020 - 2023',
    labels={'Cases': 'Total Cases'}
)
fig2.update_traces(marker_line_width=1, marker_line_color='black')
fig2.update_geos(fitbounds="locations", visible=True, showcountries=True)

# Show Actual Scale Map
fig2.show()